In [1]:
!apt update

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,821 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:13 https://developer.download

PySpark Setup

In [2]:
!apt install openjdk-8-jdk -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark



--2021-12-25 04:18:54--  https://archive.apache.org/dist/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235899716 (225M) [application/x-gzip]
Saving to: ‘spark-2.4.8-bin-hadoop2.7.tgz’

spark-2.4.8-bin-had 100%[===================>] 224.97M  87.0MB/s    in 2.6s    

2021-12-25 04:18:57 (87.0 MB/s) - ‘spark-2.4.8-bin-hadoop2.7.tgz’ saved [235899716/235899716]



## Setting Enviorment

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("pyspark Colab").config('spark.ui.port', '4050').getOrCreate()

In [4]:
spark

## Spark Dataframe frm Zero

In [11]:
from pyspark.sql import *

dept1 = Row(id="1234", name="Sistem Informasi")
dept2 = Row(id="5678", name="Teknik Informatika")

In [12]:
Student = Row('firsname', 'lastname', 'mark')
student1 = Student('achmad', 'z', 7)
student2 = Student('budi', 'yono', 8)
student3 = Student('adi', 'birowo', 8)

In [13]:
dept_si_students = Row(department=dept1, students=[student1, student3])
dept_ti_students = Row(department=dept2, students=[student2])

In [14]:
print(dept_si_students)

Row(department=Row(id='1234', name='Sistem Informasi'), students=[Row(firsname='achmad', lastname='z', mark=7), Row(firsname='adi', lastname='birowo', mark=8)])


In [16]:
print(dept_si_students.students[0].mark)

7


## Membuat Data Frame dari Rows

In [18]:
df_all = spark.createDataFrame([dept_si_students,dept_ti_students])

In [21]:
df_all.show()

+--------------------+--------------------+
|          department|            students|
+--------------------+--------------------+
|[1234, Sistem Inf...|[[achmad, z, 7], ...|
|[5678, Teknik Inf...|   [[budi, yono, 8]]|
+--------------------+--------------------+



In [22]:
df_all.show(1)

+--------------------+--------------------+
|          department|            students|
+--------------------+--------------------+
|[1234, Sistem Inf...|[[achmad, z, 7], ...|
+--------------------+--------------------+
only showing top 1 row



## Exploding Columns

In [23]:
from pyspark.sql.functions import explode

exploded_df = df_all.select(explode("students").alias("s"))

In [24]:
flatten_students_df = exploded_df.selectExpr("s.firsname", "s.lastname", "s.mark")
flatten_students_df.show()

+--------+--------+----+
|firsname|lastname|mark|
+--------+--------+----+
|  achmad|       z|   7|
|     adi|  birowo|   8|
|    budi|    yono|   8|
+--------+--------+----+



In [25]:
flatten_students_df.printSchema()

root
 |-- firsname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- mark: long (nullable = true)



## REad CSV

In [26]:
df_california = spark.read.csv('/content/sample_data/california_housing_train.csv', header=True)
df_california.show(10)
df_california.count()
df_california.printSchema()

+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|-114.310000|34.190000|         15.000000|5612.000000|   1283.000000|1015.000000| 472.000000|     1.493600|      66900.000000|
|-114.470000|34.400000|         19.000000|7650.000000|   1901.000000|1129.000000| 463.000000|     1.820000|      80100.000000|
|-114.560000|33.690000|         17.000000| 720.000000|    174.000000| 333.000000| 117.000000|     1.650900|      85700.000000|
|-114.570000|33.640000|         14.000000|1501.000000|    337.000000| 515.000000| 226.000000|     3.191700|      73400.000000|
|-114.570000|33.570000|         20.000000|1454.000000|    326.000000| 624.000000| 262.000000|     1.925000|    